# Mink usage examples

This notebook demonstrates some examples of using mink and how it interacts with sklearn.

## Imports

In [1]:
import numpy as np
from sklearn.grid_search import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.datasets import make_regression
import tensorflow as tf
import pandas as pd

In [2]:
from mink import NeuralNetClassifier
from mink import NeuralNetRegressor
from mink.layers import DenseLayer
from mink.layers import InputLayer
from mink.updates import SGD

## MNIST classification task

### Data

In [3]:
X, y = make_classification(n_samples=5000)

In [4]:
y = y.reshape(-1, 1)  # should always be 2D

### Define neural net

Note: No need to specify the shape of the training data, number of classes, or to set softmax nonlinearity. The `NeuralNetClassifier` class takes care of all of that, as is expected from an sklearn estimator.

In [5]:
l0 = InputLayer()
l1 = DenseLayer(l0, num_units=200)
l2 = DenseLayer(l1)

In [6]:
net = NeuralNetClassifier(l2, verbose=1)

If we want to change certain parameters after initialization, just use the `set_params` method and the double-underscore notation known from sklearn.

In [7]:
net.set_params(update__learning_rate=0.5)

NeuralNetClassifier(batch_iterator=<mink.nolearn.BatchIterator object at 0x7f6c74915940>,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=InputLayer(Xs=None, name=None, ys=None), name=None,
      nonlinearity=None, num_units=200),
      name=None, nonlinearity=None, num_units=None),
          max_epochs=10, objective=CrossEntropy(eps=1e-12), session=None,
          update=SGD(learning_rate=0.5), verbose=1)

### Train

In [8]:
net.fit(X, y, num_epochs=0)

NeuralNetClassifier(batch_iterator=<mink.nolearn.BatchIterator object at 0x7f6c74915940>,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=InputLayer(Xs=None, name=None, ys=None), name=None,
      nonlinearity=None, num_units=200),
      name=None, nonlinearity=Softmax(), num_units=2),
          max_epochs=10, objective=CrossEntropy(eps=1e-12), session=None,
          update=SGD(learning_rate=0.5), verbose=1)

In [9]:
y_proba = net.predict_proba(X)

In [10]:
(np.argmax(y_proba, 1) == y).mean()

0.49999399999999999

In [11]:
pdb on

Automatic pdb calling has been turned ON


In [12]:
net.fit(X, y, num_epochs=20)

epochs:    1 | loss: 0.29358
epochs:    2 | loss: 0.26525
epochs:    3 | loss: 0.24609
epochs:    4 | loss: 0.23205
epochs:    5 | loss: 0.22061
epochs:    6 | loss: 0.21174
epochs:    7 | loss: 0.20454
epochs:    8 | loss: 0.19854
epochs:    9 | loss: 0.19316
epochs:   10 | loss: 0.18838
epochs:   11 | loss: 0.18406
epochs:   12 | loss: 0.18045
epochs:   13 | loss: 0.17677
epochs:   14 | loss: 0.17347
epochs:   15 | loss: 0.17074
epochs:   16 | loss: 0.16828
epochs:   17 | loss: 0.16553
epochs:   18 | loss: 0.16287
epochs:   19 | loss: 0.16075
epochs:   20 | loss: 0.15847


NeuralNetClassifier(batch_iterator=<mink.nolearn.BatchIterator object at 0x7f6c74915940>,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=InputLayer(Xs=None, name=None, ys=None), name=None,
      nonlinearity=None, num_units=200),
      name=None, nonlinearity=Softmax(), num_units=2),
          max_epochs=10, objective=CrossEntropy(eps=1e-12), session=None,
          update=SGD(learning_rate=0.5), verbose=1)

In [13]:
y_proba

array([[ 0.58986157,  0.41013846],
       [ 0.61597109,  0.38402891],
       [ 0.3824439 ,  0.6175561 ],
       ..., 
       [ 0.30513164,  0.69486839],
       [ 0.49141669,  0.50858325],
       [ 0.35470551,  0.64529449]], dtype=float32)

In [14]:
np.argmax(y_proba, 1)[:10]

array([0, 0, 1, 0, 0, 1, 1, 1, 1, 0])

In [15]:
y_proba = net.predict_proba(X)

In [16]:
(np.argmax(y_proba, 1) == y.flatten()).mean()

0.90920000000000001

### Grid search

The neural net estimators can be used in conjunction with other sklearn features, such as `GridSearchCV`.

In [17]:
l0 = InputLayer()
l1 = DenseLayer(l0, name='hidden')
l2 = DenseLayer(l1)

In [18]:
net = NeuralNetClassifier(l2, update=SGD())

Parameters are set using the known double-underscore notation, e.g.

`'update__learning_rate': [0.1, 0.5]`.

Note: Instead of having to write

`'layer__incoming__num_units': [50, 100]`

we can just write

`'hidden__num_units': [50, 100]`

because we have given the hidden layer a name, "hidden". This may safe a lot of writing and confusion.

In [19]:
params = {
    'update__learning_rate': [0.1, 0.5],
    'max_epochs': [5, 10],
    'hidden__num_units': [50, 100],
}

In [20]:
cv = GridSearchCV(net, params, scoring='accuracy', refit=False, verbose=3)

In [21]:
cv.fit(X, y)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] update__learning_rate=0.1, max_epochs=5, hidden__num_units=50 ...
[CV]  update__learning_rate=0.1, max_epochs=5, hidden__num_units=50, score=0.854829 -   0.2s
[CV] update__learning_rate=0.1, max_epochs=5, hidden__num_units=50 ...
[CV]  update__learning_rate=0.1, max_epochs=5, hidden__num_units=50, score=0.851230 -   0.3s
[CV] update__learning_rate=0.1, max_epochs=5, hidden__num_units=50 ...
[CV]  update__learning_rate=0.1, max_epochs=5, hidden__num_units=50, score=0.849940 -   0.3s
[CV] update__learning_rate=0.5, max_epochs=5, hidden__num_units=50 ...
[CV]  update__learning_rate=0.5, max_epochs=5, hidden__num_units=50, score=0.862028 -   0.3s
[CV] update__learning_rate=0.5, max_epochs=5, hidden__num_units=50 ...
[CV]  update__learning_rate=0.5, max_epochs=5, hidden__num_units=50, score=0.863827 -   0.4s
[CV] update__learning_rate=0.5, max_epochs=5, hidden__num_units=50 ...
[CV]  update__learning_rate=0.5, max_epochs=5, hi

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:   16.5s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=NeuralNetClassifier(batch_iterator=<mink.nolearn.BatchIterator object at 0x7f6c74915940>,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=DenseLayer(W=Glor...bjective=CrossEntropy(eps=1e-12), session=None,
          update=SGD(learning_rate=0.01), verbose=0),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_epochs': [5, 10], 'update__learning_rate': [0.1, 0.5], 'hidden__num_units': [50, 100]},
       pre_dispatch='2*n_jobs', refit=False, scoring='accuracy', verbose=3)

In [22]:
cv.best_params_, cv.best_score_

({'hidden__num_units': 50, 'max_epochs': 10, 'update__learning_rate': 0.5},
 0.87380000000000002)

## Regression task

As is known from sklearn, we have separate estimators for classification and regression.

### Data

In [23]:
X, y = make_regression(n_samples=5000)

In [24]:
y = y.reshape(-1, 1)  # should always be 2D

Note that apart from using `NeuralNetRegressor` instead of `NeuralNetClassifier`, everything is the same. No need to adjust output nonlinearity or objective.

### Define neural network

In [25]:
l0 = InputLayer()
l1 = DenseLayer(l0, num_units=200)
l2 = DenseLayer(l1)

In [26]:
net = NeuralNetRegressor(l2, verbose=1)

In [27]:
net.set_params(update__learning_rate=0.001)

NeuralNetRegressor(batch_iterator=<mink.nolearn.BatchIterator object at 0x7f6c74915d30>,
          encoder=None,
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=InputLayer(Xs=None, name=None, ys=None), name=None,
      nonlinearity=None, num_units=200),
      name=None, nonlinearity=None, num_units=None),
          max_epochs=10, objective=MeanSquaredError(), session=None,
          update=SGD(learning_rate=0.001), verbose=1)

### Train

In [28]:
net.fit(X, y, num_epochs=20)

epochs:    1 | loss: 3543.69019
epochs:    2 | loss: 230.67401
epochs:    3 | loss: 173.87079
epochs:    4 | loss: 131.61575
epochs:    5 | loss: 98.18998
epochs:    6 | loss: 72.82103
epochs:    7 | loss: 53.76473
epochs:    8 | loss: 39.89501
epochs:    9 | loss: 29.63145
epochs:   10 | loss: 22.29911
epochs:   11 | loss: 16.98507
epochs:   12 | loss: 12.98714
epochs:   13 | loss: 10.10332
epochs:   14 | loss: 8.02698
epochs:   15 | loss: 6.54984
epochs:   16 | loss: 5.47682
epochs:   17 | loss: 4.66838
epochs:   18 | loss: 4.05618
epochs:   19 | loss: 3.58627
epochs:   20 | loss: 3.20913


NeuralNetRegressor(batch_iterator=<mink.nolearn.BatchIterator object at 0x7f6c74915d30>,
          encoder=None,
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=InputLayer(Xs=None, name=None, ys=None), name=None,
      nonlinearity=None, num_units=200),
      name=None, nonlinearity=Linear(), num_units=1),
          max_epochs=10, objective=MeanSquaredError(), session=None,
          update=SGD(learning_rate=0.001), verbose=1)